<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/real_fake_news_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"lmanqasml","key":"2d851a4eb9cae06770577185722326e0"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download razanaqvi14/real-and-fake-news

Dataset URL: https://www.kaggle.com/datasets/razanaqvi14/real-and-fake-news
License(s): CC-BY-NC-SA-4.0
  0% 0.00/41.0M [00:00<?, ?B/s]
100% 41.0M/41.0M [00:00<00:00, 561MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/real-and-fake-news.zip', 'r')
zip_ref.extractall()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_true=pd.read_csv('/content/True.csv')

In [ ]:
df_fake=pd.read_csv('/content/Fake.csv')

In [ ]:
df_true["label"] = 1
df_fake["label"] = 0

In [ ]:
df = pd.concat([df_true, df_fake], ignore_index=True)

In [ ]:
df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

df['text'] = df['text'].apply(clean_text)
df['title']=df['title'].apply(clean_text)

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word,pos="v") for word in words]
    return ' '.join(lemmatized_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df['text'] = df['text'].apply(lemmatize_text)
df['title']=df['title'].apply(lemmatize_text)

In [ ]:
df.head(7)

,title,text,subject,date,label
0,as us budget fight loom republicans flip their...,washington reuters the head of a conservative ...,politicsNews,"December 31, 2017",1
1,us military to accept transgender recruit on m...,washington reuters transgender people will be ...,politicsNews,"December 29, 2017",1
2,senior us republican senator let mr mueller do...,washington reuters the special counsel investi...,politicsNews,"December 31, 2017",1
3,fbi russia probe help by australian diplomat t...,washington reuters trump campaign adviser geor...,politicsNews,"December 30, 2017",1
4,trump want postal service to charge much more ...,seattlewashington reuters president donald tru...,politicsNews,"December 29, 2017",1
5,white house congress prepare for talk on spend...,west palm beach flawashington reuters the whit...,politicsNews,"December 29, 2017",1
6,trump say russia probe will be fair but timeli...,west palm beach fla reuters president donald t...,politicsNews,"December 29, 2017",1


In [ ]:
import tensorflow as tf
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=1000, output_sequence_length=50)

In [ ]:
text_vectorizer.adapt(df['text'])

In [ ]:
text_vectorizer.adapt(df['title'])

In [ ]:
df['combined'] = df['title'] + ' ' + df['text']

In [ ]:
X = df['combined']
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
seq_lengths = X_train.apply(lambda x: len(x.split()))

# Statistikaya bax
print("Min length:", seq_lengths.min())
print("Max length:", seq_lengths.max())
print("Average length:", seq_lengths.mean())
print("95th percentile:", np.percentile(seq_lengths, 95))

Min length: 0
Max length: 8133
Average length: 415.27239824043653
95th percentile: 909.0


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000, output_sequence_length=100)
text_vectorizer.adapt(X_train)

In [ ]:
from collections import Counter

# Bütün sözləri birləşdir
all_words = ' '.join(X_train).split()
word_counts = Counter(all_words)

print("Unique words:", len(word_counts))

Unique words: 185658


In [ ]:
# Tensorflow dataset yaradılır
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:

embedding_dim = 64

model = tf.keras.Sequential([
    text_vectorizer,
    tf.keras.layers.Embedding(input_dim=10000, output_dim=embedding_dim, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=test_ds, epochs=5)


Epoch 1/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 187s 160ms/step - accuracy: 0.9434 - loss: 0.1156 - val_accuracy: 0.9974 - val_loss: 0.0132
Epoch 2/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.9992 - loss: 0.0040 - val_accuracy: 0.9974 - val_loss: 0.0119
Epoch 3/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 199s 177ms/step - accuracy: 0.9998 - loss: 0.0013 - val_accuracy: 0.9932 - val_loss: 0.0470
Epoch 4/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 183s 163ms/step - accuracy: 0.9997 - loss: 0.0013 - val_accuracy: 0.9965 - val_loss: 0.0226
Epoch 5/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 182s 162ms/step - accuracy: 1.0000 - loss: 1.2015e-04 - val_accuracy: 0.9971 - val_loss: 0.0328
